In [ ]:
# Notebook created to test SPhericity class implemented in numpythia

In [1]:
import numpy as np
import sys
import dimod
import matplotlib.pyplot as plt
import os
import re
import neal

from dwave.system import EmbeddingComposite, DWaveSampler, ReverseAdvanceComposite
from dwave_qbsolv import QBSolv
from pyqubo import Spin, Array
from embedding_generator import generate_embedding

In [2]:
path = '/Users/1zd/quantumjets/pythiaEvents/'

theta = np.pi/2.

sampler = EmbeddingComposite(DWaveSampler(solver='Advantage_system1.1'))
sa_sampler = neal.SimulatedAnnealingSampler()

In [3]:
fname = path + 'Event_0.dat'
f = open(fname)

pat = re.compile(r"\s+")

g = list(zip(*[ pat.sub(" ",x.strip()).split() for x in f]))

px = np.array([ float(x) for x in g[0]])
py = np.array([ float(x) for x in g[1]])
pz = np.array([ float(x) for x in g[2]])
e = np.array([ float(x) for x in g[3]])

# Generate qubo

n_part = len(px)
#print(n_part)
s = Array.create('s', shape=n_part, vartype='BINARY')

# Create an array of zeroes for qubo matrix coefficients
coeff = [[0] * n_part for _ in range(n_part)] 

for i in range(0,n_part):
    for j in range(0,n_part):
        coeff[i][j] = (px[i]*px[j] +py[i]*py[j] + pz[i]*pz[j] - e[i]*e[j] * np.cos(theta))/(1 - np.cos(theta))

        # Construct Hamiltonian
        #H = sum([ ((1.0/(np.sqrt(px[i]*px[i] +py[i]*py[i] + pz[i]*pz[i])*np.sqrt(px[j]*px[j] +py[j]*py[j] + pz[j]*pz[j]))) - 1.0)*coeff[i][j]*s[i]*s[j] for i in range(0,n_part) for j in range(0,n_part)])
H = sum([ -1.0*coeff[i][j]*s[i]*s[j] for i in range(0,n_part) for j in range(0,n_part)])

        # Compile model using pyqubo

model = H.compile()
qubo, offset = model.to_qubo()

        # Submit problem to D-Wave
        #response = QBSolv().sample_qubo(qubo)#
response = sampler.sample_qubo(qubo,num_reads=100, annealing_time=20)

In [4]:
response.samples

<bound method SampleSet.samples of SampleSet(rec.array([([1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1], -2012.00503893, 3, 0.),
           ([1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1], -2006.79808284, 1, 0.),
           ([1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1], -2005.20079092, 1, 0.),
           ([1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1], -2004.28970445, 2, 0.),
           ([0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0], -2002.4059079 , 1, 0.),
           ([1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1], -2002.40391023, 2, 0.),
           ([1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1], -2000.79770587, 1, 0.),
           ([1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1], -19